# pdf2reveal
This notebook transforms a presentation PDF made using LibreOffice Impress, Microsoft Powerpoint or Latex Beamer into HTML (reveal.js). This allows displaying the presentation on any device running a web browser. 

I wrote this so that I can present lecture slides on a Promethean without having to connect my computer. 

Files and directories created by this notebook:
* template.html - a jinja2 template
* ../images (dir)
* 'images/ent_101_mimicry-000.jpg' ... page images extracted from the PDF
* index.html - this is the reveal.js presentation

This notebook reads a PDF file, creates an image directory

### Technical details

###### Initialization

During the initial run of this notebook, the **install_reveal** function installed the latest version of **reveal.js** which was **4.1.3**. The **create_template** function then created a **jinja2** template by editing 
**reveal.js-master/index.html**. Note that my notebook code may break if a later version of **reveal.js** is installed.

##### Fixing imagemagick problems
This notebook uses the **convert** command, which is part of **imagemagick** to convert PDFs to images.
Had some initial problems which were fixed by changing a few lines in the config file **/etc/ImageMagick-6/policy.xml**:
```
<policy domain="coder" rights="none" pattern="PDF" />
<policy domain="resource" name="memory" value="256MiB"/>
<policy domain="resource" name="disk" value="1GiB"/>
```
where changed to:
```
<!--  <policy domain="coder" rights="none" pattern="PDF" />  -->
<policy domain="resource" name="memory" value="8GiB"/>
<policy domain="resource" name="disk" value="8GiB"/>
```
### References
* https://marcus-baw.medium.com/using-reveal-js-a74b30e4065b
* https://martinomensio.medium.com/how-to-host-reveal-js-slides-on-github-pages-and-have-a-tidy-repository-1a363944c38d
* https://superuser.com/questions/1178666/imagemagick-convert-quits-after-some-pages

In [20]:
import os
import requests, zipfile, io
import jinja2
import glob
import shutil

In [21]:
PDFPATH = '../ent_101_mimicry.pdf'

In [22]:
def create_template():
    """
    Create a jinja2 template by editing the reveal.js default index.html
    """

    # Read the default index.html as a string    
    s = open('../index.html', 'r').read()

    # edit the string and save it as template.html
    s = s.replace('<section>Slide 1</section>\n\t\t\t\t<section>Slide 2</section>',
              '{% for image in images %}\n\t\t\t\t\t<section> <img src="{{ image }}"> </section>\n\t\t\t\t{% endfor %}')

    with open('template.html', 'w') as f:
        f.write(s)

In [23]:
def create_images():
    """
    Convert the PDF pages into jpg images and store them in the images directory
    """
    if not os.path.exists('../images'):
        os.mkdir('../images') 
    pdf_name = os.path.basename(PDFPATH).replace('.pdf','')
    command = f'convert -density 300 {PDFPATH} ../images/{pdf_name}-%03d.jpg'
    print(command)
    os.system(command)

In [24]:
def create_presentation():
    """
    Use the jinja2 template to add slides from the images directory    
    """
    templateLoader = jinja2.FileSystemLoader(searchpath="./")
    templateEnv = jinja2.Environment(loader=templateLoader)
    TEMPLATE_FILE = "template.html"
    template = templateEnv.get_template(TEMPLATE_FILE)
    
    pdf_name = os.path.basename(PDFPATH).replace('.pdf','')
    imagelist = sorted(glob.glob(f'../images/{pdf_name}-???.jpg')) 
    for i, image in enumerate(imagelist):
        imagelist[i] = image.replace('../','')        
    print(imagelist)
    outputText = template.render({'images': imagelist})

    # Overwrite the original index.html file.
    # Open this file in a browser to see the reveal.js presentation.
    with open(f'../index.html', 'w') as out:
        out.write(outputText)

In [25]:
# MAIN
create_template()
create_images()
create_presentation()

convert -density 300 ../ent_101_mimicry.pdf ../images/ent_101_mimicry-%03d.jpg
['images/ent_101_mimicry-000.jpg', 'images/ent_101_mimicry-001.jpg', 'images/ent_101_mimicry-002.jpg', 'images/ent_101_mimicry-003.jpg', 'images/ent_101_mimicry-004.jpg', 'images/ent_101_mimicry-005.jpg', 'images/ent_101_mimicry-006.jpg', 'images/ent_101_mimicry-007.jpg', 'images/ent_101_mimicry-008.jpg', 'images/ent_101_mimicry-009.jpg', 'images/ent_101_mimicry-010.jpg', 'images/ent_101_mimicry-011.jpg', 'images/ent_101_mimicry-012.jpg', 'images/ent_101_mimicry-013.jpg', 'images/ent_101_mimicry-014.jpg', 'images/ent_101_mimicry-015.jpg', 'images/ent_101_mimicry-016.jpg', 'images/ent_101_mimicry-017.jpg', 'images/ent_101_mimicry-018.jpg', 'images/ent_101_mimicry-019.jpg', 'images/ent_101_mimicry-020.jpg', 'images/ent_101_mimicry-021.jpg', 'images/ent_101_mimicry-022.jpg', 'images/ent_101_mimicry-023.jpg', 'images/ent_101_mimicry-024.jpg', 'images/ent_101_mimicry-025.jpg', 'images/ent_101_mimicry-026.jpg', 'i